#Face Mask Detection using yolo V3 

**1. Check the connected GPU version**

In [1]:
from IPython.display import HTML, clear_output
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:gpu = 'K80'
elif 'T4' in s:gpu = 'T4'
elif 'P100' in s:gpu = 'P100'
elif 'P4' in s:gpu = 'P4'
display(HTML(f"<h1>{gpu}</h1>"))

**2. Mount the Drive**

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


**3. Clone the Darknet repository**

In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15298, done.
remote: Total 15298 (delta 0), reused 0 (delta 0), pack-reused 15298
Receiving objects: 100% (15298/15298), 13.68 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (10398/10398), done.


**4. Compile Darknet**

In [ ]:
%cd /content/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**5. Create a copy of yolov3.cfg file to avoid changes in original file.**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

**6. configuring the yolov3_training.cfg**<br>
**classes = 2**
<br>
**filters= (classes + 5)*3 = 21**


In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg


**7. Creating a classes file**

In [7]:
!echo -e 'wearing Mask\nNot Wearing Mask' >data/obj.names

**8. Creating a object data file**

In [8]:
!echo -e 'classes=2\ntrain = data/train.txt\nvalid=data/test.txt\nnames=data/obj.names\nbackup =  /content/drive/MyDrive/Object_Detection_yoloV3' > data/obj.data

**9. Unzip the data into data/obj**

In [ ]:
!mkdir data/obj
!unzip /content/drive/MyDrive/images.zip -d data/obj



**Rename the 'data/obj/New folder' to 'data/obj/images'**

In [10]:
import glob
images_list = glob.glob("/content/darknet/data/obj/images/*.png")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**10. Download pre-trained weights for Convolutional layer file**

In [14]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-08-12 15:29:31--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.5MB/s    in 2.7s    

2021-08-12 15:29:34 (56.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**11. Train the model**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg  /content/drive/MyDrive/Object_Detection_yoloV3/yolov3_training_last.weights -dont_show

**11. mask detection from a video**<br>
**Detects a mask wearing and not wearing faces from a video and store it in google drive.**

In [ ]:
!./darknet detector demo  data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/Object_Detection_yoloV3/yolov3_training_last.weights -dont_show  /content/video1.mp4 -i 0 -out_filename /content/drive/MyDrive/Object_Detection_yoloV3/predictions.avi

**12. Mask detection from Images**

In [15]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
!./darknet detector test data/obj.data /content/darknet/cfg/yolov3_training.cfg  /content/drive/MyDrive/Object_Detection_yoloV3/yolov3_training_last.weights /content/without-mask-UNI-3.jpg
imShow('predictions.jpg')
